In [1]:
from imprint.nb_util import setup_nb
setup_nb()
import asyncio
import redis
import numpy as np

import confirm.cloud.clickhouse as ch

from confirm.adagrid.validate import AdaValidate
from confirm.adagrid.calibrate import AdaCalibrate
from confirm.adagrid.validate import ada_validate
from confirm.adagrid.calibrate import ada_calibrate
from confirm.adagrid.redis_heartbeat import HeartbeatThread
import confirm.adagrid.adagrid as adagrid
import imprint as ip
from imprint.models.ztest import ZTest1D

import clickhouse_connect
clickhouse_connect.common.set_setting('autogenerate_session_id', False)

### Glossary

- **coordination**: All workers come together and divide up all tiles.
- **step**: A worker checks the convergence criterion, select tiles, and simulates.
- **packet**: The unit of simulation work that a worker requests from the DB.
- **batch**: The unit of simulation work that is passed to a Model.

In [7]:
import time
start = time.time()
db = ch.Clickhouse.connect()
print('1', time.time() - start)
g = ip.cartesian_grid(
    theta_min=[-1], theta_max=[1], n=[50000], null_hypos=[ip.hypo("x0 < 0")]
)
print('2', time.time() - start)

import inspect
sig = inspect.signature(ada_calibrate)
kwargs={k: v.default for k,v in sig.parameters.items()}
kwargs.update(dict(lam=-1.96, prod=False))
n_workers=2
ada = adagrid.Adagrid()
print('3', time.time() - start)
await ada.init(ZTest1D, g, db, AdaCalibrate, print, None, n_workers, kwargs, worker_id = 1)
print('4', time.time() - start)

INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/None
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Clickhouse config: default@t2n4z83v82.us-east-1.aws.clickhouse.cloud:8443/18e078df456241bb939f2f95130e0806
INFO:confirm.cloud.clickhouse:[worker_id=None] 
Connected to job 18e078df456241bb939f2f95130e0806
DEBUG:imprint.grid:[worker_id=None] 
_gen_short_uuids(n=50000, worker_id=1, t=1676305046, n_bits=18, worker_bits=18) = [4514341168779362304 4514341168779362305 4514341168779362306, ...]:


1 0.43687987327575684
2 0.4547920227050781
3 0.45650386810302734
11 5.0067901611328125e-06
12 2.193450927734375e-05
13 4.00543212890625e-05
14 0.04837799072265625
14.5 0.04994797706604004
15 0.05003213882446289
16 0.05087995529174805
17 0.21451783180236816
18 0.21810603141784668
waiting
waiting
waiting


DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Initialized database with 25000 tiles and 1 null hypos. The tiles are split between 2 workers with packet_size=1024.


19 0.22108697891235352


DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
set_or_append(config) -> False None
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
set(config) -> False config


20 1.5717859268188477


DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
set(null_hypos) -> False null_hypos
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
writing 25000 tiles


4 3.8560729026794434


In [3]:
async with HeartbeatThread(ada.db.redis_con, ada.db.job_id, ada.worker_id):
    await ada.process_step(0)

DEBUG:confirm.adagrid:[worker_id=1] 
Acquired lock 30a3380ae51040f192bafd4825926d41:heartbeat_lock_1
DEBUG:confirm.adagrid:[worker_id=1] 
Added worker 1 to workers set.
DEBUG:confirm.adagrid:[worker_id=1] 
Extended lock 30a3380ae51040f192bafd4825926d41:heartbeat_lock_1 for 30 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Simulated 1024 tiles in 0.20 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Simulated 1024 tiles in 0.01 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Simulated 1024 tiles in 0.01 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Simulated 1024 tiles in 0.01 seconds.
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Simulated 1024 tiles in 0.01 seconds.
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
writing 1024 results
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Simulated 1024 tiles in 0.02 seconds.
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
writing 1024 results
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Simulated 1024 tiles in 0.01 seconds.

In [14]:
report = dict()
await ada.new_step(1, report)

INFO:confirm.adagrid.validation:[worker_id=1] 
Preparing new step with 1024 parent tiles.
DEBUG:imprint.grid:[worker_id=1] 
_gen_short_uuids(n=1024, worker_id=1, t=1676091287, n_bits=18, worker_bits=18) = [4499651762151751680 4499651762151751681 4499651762151751682, ...]:
DEBUG:imprint.grid:[worker_id=1] 
_gen_short_uuids(n=0, worker_id=1, t=1676091288, n_bits=18, worker_bits=18) = [, ...]:
DEBUG:confirm.adagrid.adagrid:[worker_id=1] 
Starting step 1 with 1024 tiles to simulate.
DEBUG:confirm.cloud.clickhouse:[worker_id=1] 
finish:                     id  step_id  packet_id  active  finisher_id  refine  \
0  4499608056564560715        0          6   False            1   False   
1  4499608056564560717        0          6   False            1   False   
2  4499608056564560720        0          6   False            1   False   
3  4499608056564560722        0          6   False            1   False   
4  4499608056564560723        0          6   False            1   False   

   deepen  

(<WorkerStatus.NEW_STEP: 5>,
 {'converged': False,
  'max_tie_est': 0.0076904297,
  'next_tile_tie_est': 0.0076904296875,
  'next_tile_tie_bound': 0.01024944894015789,
  'next_tile_sim_cost': 0.0025583983563868366,
  'next_tile_grid_cost': 6.208962710536892e-07,
  'next_tile_total_cost': 0.0025590192526578903,
  'next_tile_K': 8192.0,
  'next_tile_at_max_K': False,
  'runtime_convergence_criterion': 0.1363811492919922,
  'n_tiles': 1024,
  'step_max_total_cost': 0.0025590192526578903,
  'step_max_grid_cost': 6.208962710536892e-07,
  'step_max_sim_cost': 0.0025583983563868366,
  'runtime_select_tiles': 0.1318209171295166,
  'n_refine': 0,
  'n_deepen': 1024,
  'n_complete': 0})

In [15]:
ada.db.get_tiles()

,id,parent_id,theta0,radii0,null_truth0,K,step_id,creator_id,creation_time,worker_id,packet_id,active
0,4499608056564547584,0,-0.99998,0.00002,1,8192,0,1,1.676091e+09,1,0,1
1,4499608056564547586,0,-0.99990,0.00002,1,8192,0,1,1.676091e+09,1,0,1
2,4499608056564547589,0,-0.99978,0.00002,1,8192,0,1,1.676091e+09,1,0,1
3,4499608056564547594,0,-0.99958,0.00002,1,8192,0,1,1.676091e+09,1,0,1
4,4499608056564547597,0,-0.99946,0.00002,1,8192,0,1,1.676091e+09,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
29045,4499651762151752699,4499608056564558904,-0.54718,0.00002,1,16384,1,1,1.676091e+09,1,0,1
29046,4499651762151752700,4499608056564558905,-0.54714,0.00002,1,16384,1,1,1.676091e+09,1,0,1
29047,4499651762151752701,4499608056564558907,-0.54706,0.00002,1,16384,1,1,1.676091e+09,1,0,1
29048,4499651762151752702,4499608056564558908,-0.54702,0.00002,1,16384,1,1,1.676091e+09,1,0,1


In [ ]:
report

In [45]:
df = ada.db.get_results()

In [46]:
df['packet_id'].max()

12

In [51]:
ada.db.get_reports().sort_values(by=['worker_id', 'step_id', 'packet_id'])

,worker_id,step_id,packet_id,n_tiles,runtime_get_work,runtime_process_tiles,runtime_insert_results,status,runtime_done_wait,runtime_skip_packet
0,1,0,0,1024,0.116129,0.058329,0.000013,WORK,NaN,NaN
15,1,0,0,1024,NaN,NaN,NaN,SKIPPED,NaN,0.138124
30,1,0,0,1024,NaN,NaN,NaN,SKIPPED,NaN,0.108462
2,1,0,1,1024,0.059154,0.011068,0.000011,WORK,NaN,NaN
16,1,0,1,1024,NaN,NaN,NaN,SKIPPED,NaN,0.049365
31,1,0,1,1024,NaN,NaN,NaN,SKIPPED,NaN,0.078803
1,1,0,2,1024,0.057187,0.011943,0.000011,WORK,NaN,NaN
17,1,0,2,1024,NaN,NaN,NaN,SKIPPED,NaN,0.048913
32,1,0,2,1024,NaN,NaN,NaN,SKIPPED,NaN,0.058951
4,1,0,3,1024,0.056851,0.015085,0.000013,WORK,NaN,NaN
